# TF DEV Summit - TF.DATA

 - Modu Labs - 신성진 by. Deep NLP

# TF.DATA를 시작하기 전에 알아야 할 개념!

## Background - Dynamic vs. Static

### Dynamic 방식으로 데이터를 처리하면, 모든 데이터를 메모리에 올리지 않고, 필요할 때만 추출한다. tf.data를 시작하기 전에 iterator / Generator / Yield에 대해 이해하고 있어야 활용이 용이하다.

- http://stackabuse.com/python-generators/ 

- http://pymbook.readthedocs.io/en/latest/igd.htm

- https://dojang.io/mod/page/view.php?id=1117

 1. Iterator - Repeatable Object (반복가능한 객체)

 2. Generator - Iterator를 만들어주는 것, lazy generation of values (on-demand)
 
 3. Yield - 함수에서 return과 동일한 역할 수행

### Adventages

- 온디멘드 방식으로 메모리를 적게 사용 (Streaming, Big Data)

- 사용하지 않는 값들은 저장하고 있지 않음

### Disadventages

- Static 방식에 비해 로드 때마다 IO 연산이 발생되는데, 이후 TF.DATA는 관련 방식을 어떻게 해결하는지 확인해보자

In [3]:
# Generator 활용의 예

#yield: 함수 실행 중간에 빠져나올 수 있는 generator를 만들 때 사용

def num_gen(n):
    num = 0
    while num < n:
        yield num # loop 중간에 값을 가져옴
        num += 1

gen_execute = num_gen(10)

print("Dynamic (동적) 접근")
print(next(gen_execute))
print(next(gen_execute))
# 위와 같은 방식으로 데이터를 필요 할 때만 가져오는 방식으로 접근한다.

# Static으로 변환하고 싶으면 list를 활용하자
print("Static(정적)변환 {}".format(list(num_gen(10))))

Dynamic (동적) 접근
0
1
Static(정적)변환 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


# TF.Data

## 머신러닝의 시작은 결국 데이터를 어떻게 다루느냐가 Key 요소 중 하나.

## 단순히 연구자 뿐만 아니라, 상용화 (Product)를 위해서 끊임없이 개선하고 있음.

### **The tf.data mission**

Input piplines for Tensorflow should be:

**Fast** : to keep up with GPUs and TPUs

**Flexible** : to handle diverse data sources and use cases

**Easy to use** : to democratize machine learning

### Extract Transform Load for Tensorflow

#### data pipeline을 구축함으로써, 학습 데이터 준비와 학습 시간 최적화 - 1

<img src="tf.data1.png">

```python
#Extract
files = tf.data.Dataset.list_files(file_pattern)
dataset = tf.data.TFRecordDataset(files)

#Transform
dataset = dataset.shuffle(10000)
dataset = dataset.repeat(NUM_EPOCHS)
dataset = dataset.map(lambda x: tf.parse_single_example(x, features))
dataset = dataset.batch(BATCH_SIZE)

#Load
iterator = dataset.make_one_shot_iterator() #Sequencial Access
features = iterator.get_next()
```

### Performance

- CNN benchmarks reach > **13,000 images/second** with tf.data -> 8달 전과 비교하여 성능 2배 성장 (DGX - Imagenet)
 <br />
- 텐서플로우 벤치마크 프로젝트를 사용하여 활용: www.tensorflow.org/performance/datasets_performance

- New tf.contrib.data.prefetch_to_device() for GPUs tf 1.8 (tf-nightly에 보유)

- https://www.tensorflow.org/versions/master/performance/datasets_performance

#### parallel 기능을 활용하여, 학습 데이터 준비와 학습 시간 최적화 - 2

<img src="tf.data2.png">


```python

## 속도를 올리기 위해서는?? Parallel!!
## 기존에 shuffle / batch 등등 각각 진행하던 방식을 1개로 합침

#Extract
files = tf.data.Dataset.list_files(file_pattern)
# dataset = tf.data.TFRecordDataset(files) ->
dataset = tf.data.TFRecordDataset(files, num_parallel_reads=32)

#Transform

#shuffle_and_repeat: epochs와 buffers사이에서 정지하는 현상 방지
dataset = dataset.apply(
    tf.contrib.data.shuffle_and_repeat(10000, NUM_EPOCHS))

#map_and_batch: map과 data transfer를 동시에 함
dataset = dataset.apply(
    tf.contrib.data.map_and_batch(lambda x: ..., BATCH_SIZE))

#Load

#prefetch_to_device = 그 다음 batch가 미리 GPU 메모리 대기
dataset = dataset.apply(tf.contrib.data.prefetch_to_device("/gpu:0"))
iterator = dataset.make_one_shot_iterator() #Sequencial Access
features = iterator.get_next()

```

### Flexibility

- tf.SparseTensor를 지원 (1.5ver) -> 복잡한 Categorical data나 embedding 모델을 다룰때

- Custom Python code via Dataset.from_generator() ->?

- Custom C++ code via DatasetOpKernel plugis

-> 실무 새로운 데이터셋을 만들거나 개선할때 좋다.

### Easy of Use

**데이터를 읽고 쓰는 것을 조금 더 쉽게 지원하기 위한 기능 추가**

#### Use Python for loops in eager exeuction mode

```python
#Extract
files = tf.data.Dataset.list_files(file_pattern)
dataset = tf.data.TFRecordDataset(files)

#Transform
dataset = dataset.shuffle(10000)
dataset = dataset.repeat(NUM_EPOCHS)
dataset = dataset.map(lambda x: tf.parse_single_example(x, features))
dataset = dataset.batch(BATCH_SIZE)

#Eager execution make datset a normal Python iterable.

for batch in dataset:
    train_model(batch)
```

</br>

#### Standard Method CSV file with protocal buffer (tf 1.8)

```python
tf.enable_eager_execution()

#make_batched_features_dataset
dataset = tf.contrib.data.make_batched_features_dataset(
    file_pattern, BATCH_SIZE, features, num_epochs=NUM_EPOCHS)

#일반적으로는 속도를 위해서는 tf.example, tfrecord와 같은 binary를 추천
#하지만 큰 데이터를 항상 가지고 있는 것은 아님.

#kaggle의 예
#$ pip install kaggle
#$ kaggle datasets downaload -d theronk/million-headlins -p .

for batch in dataset:
    train_model(batch["publish_data"], batch["headline_text"])

```

### Integration with Esitmators(and Keras comming soon!!)

```python
def input_fn():
    dataset = tf.contrib.data.make_csv_dataset(
        "*.csv", BATCH_SIZE, num_epochs=NUM_EPOCHS)
    return dataset

# train an estimator on the dataset
tf.esitmator.Estimator(model_fn=train_model).train(input_fn=input_fn)

```

*공식 홈페이지에서의 추가 정보들과 Performance 참고*

- www.tensorflow.org/programmers_guide/datasets
- www.tensorflow.org/performance/datasets_performance

# 실습 및 활용

ref: https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428

1. Importing Data: 데이터셋 생성
 - From numpy
 - From tensor
 - From a placeholder
 - From generator
 
 </br>

2. Create an Iterator: 생성된 데이터셋을 바탕으로 Iterator 인스턴스를 만들기
 - One shot Iterator
 - Initializable Iterator
 - Reinitializable Iterator
 - Feedable Iterator
 
 </br>

In [2]:
import tensorflow as tf
import numpy as np

#with keras

/Users/sinseongjin/tf17_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 1. Importing Data

In [8]:
#numpy에서 데이터 불러오기

x = np.random.sample((300,2))
print("size of x:", x.shape)

# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    #Session을 통하여 데이터를 print해 볼 수 있음
    print(sess.run(el))

size of x: (300, 2)
[0.18595173 0.27372692]


In [13]:
# features와 label을 불러오는 경우
features, labels = (np.random.sample((100,2)), np.random.sample((100,1)))
dataset = tf.data.Dataset.from_tensor_slices((features,labels))

# tfrecords를 활용한다면, 따로 데이터와 label을 나눌 필요가 없음

In [14]:
# Tensor Data 처리
dataset = tf.data.Dataset.from_tensor_slices(tf.random_uniform([100, 2]))

In [16]:
# Placeholder 처리방법
x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)

In [17]:
# generator 활용하여 처리
# generator를 활용하여 init하기
# 주로 sequence와 같은 길이가 다른 element들이 있을때 유용

sequence = np.array([[1],[2,3],[3,4]])

def generator():
    for el in sequence:
        yield el
        
dataset = tf.data.Dataset().from_generator(generator,
                                           output_types=tf.float32, 
                                           output_shapes=[tf.float32])

## 2. Create an Iterator - get data

### One shot Iterator

In [18]:
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)

# create the iterator
iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el)) # output: [ 0.42116176  0.40666069]

[0.46524592 0.91114116]


### Initalizable Iterator

 - Dynamic dataset을 다루기 위해 placeholder와 함께 활용
 - placeholder를 feed-dict 메커니즘을 활용하여 초기화 함

In [21]:
# using a placeholder
x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)
data = np.random.sample((100,2))

iter = dataset.make_initializable_iterator() # create the iterator
el = iter.get_next()

with tf.Session() as sess:
    # feed the placeholder with data
    sess.run(iter.initializer, feed_dict={ x: data }) #initializer를 선언
    print(sess.run(el)) # output [ 0.52374458  0.71968478]

[0.9624355 0.6523885]


In [24]:
## train 과 test 데이터셋 동시에 다루기

train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.array([[1,2]]), np.array([[0]]))

EPOCHS = 10

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])

dataset = tf.data.Dataset.from_tensor_slices((x, y))
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.array([[1,2]]), np.array([[0]]))
iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

with tf.Session() as sess:
    
    #     initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1]})
    for _ in range(EPOCHS):
        sess.run([features, labels])

    #     switch to test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1]})
    print(sess.run([features, labels]))

[array([1., 2.], dtype=float32), array([0.], dtype=float32)]


### Reinitializable Iterator

 - 위의 Initalizable Iterator와 유사하지만, 새로운 데이터를 feed 하는 대신, 새로운 데이터로 변경 할 수 있음.

In [27]:
# making fake data using numpy
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))

# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)

# 한가지 트릭으로, Generic Iterator를 생성한다.

# shape와 type으로 iterator를 생성하고,
iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
# 두개를 동시에 초기화한다.

# create the initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)

# We get the next element as before
features, labels = iter.get_next()

# session을 활용하여 2개의 초기화 연산을 실행한다.

train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)

with tf.Session() as sess:
    sess.run(train_init_op) # switch to train dataset
    for _ in range(EPOCHS):
        sess.run([features, labels])
    sess.run(test_init_op) # switch to val dataset
    print(sess.run([features, labels]))

[array([0.13652812, 0.69115812]), array([0.24063632])]


## 3. Consuming data
  
  - 데이터를 모델에 pass하기 위하여 get_next() 활용

In [28]:
EPOCHS = 10
BATCH_SIZE = 16
# using two numpy arrays
features, labels = (np.array([np.random.sample((100,2))]), 
                    np.array([np.random.sample((100,1))]))

dataset = tf.data.Dataset.from_tensor_slices((features,labels)).repeat().batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator() #iterator생성하기
#첫번째 layer와 label에 iter.get_next를 통해 직접 Tensor를 넣는다.
x, y = iter.get_next()

# 간단한 뉴럴넷 만들기

# pass the first value from iter.get_next() as input
net = tf.layers.dense(x, 8, activation=tf.tanh) 
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

# pass the second value from iter.get_net() as label
loss = tf.losses.mean_squared_error(prediction, y)
train_op = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(EPOCHS):
        _, loss_value = sess.run([train_op, loss])
        print("Iter: {}, Loss: {:.4f}".format(i, loss_value))

Iter: 0, Loss: 0.1603
Iter: 1, Loss: 0.1561
Iter: 2, Loss: 0.1521
Iter: 3, Loss: 0.1482
Iter: 4, Loss: 0.1446
Iter: 5, Loss: 0.1412
Iter: 6, Loss: 0.1380
Iter: 7, Loss: 0.1351
Iter: 8, Loss: 0.1323
Iter: 9, Loss: 0.1296


# TIPS: BATCHING / REPEAT / MAP / SHUFFLE

## Batch
 - 일반적으로 복잡한 Batch의 처리를 간단하게 활용

In [10]:
# BATCHING
BATCH_SIZE = 4
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x).batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

[[0.59718665 0.89988339]
 [0.88266584 0.91656019]
 [0.84153486 0.9416401 ]
 [0.41580571 0.67149192]]


## Repeat

- .repeat()을 통해, 데이터를 반복한다.

In [34]:
# REPEAT
BATCH_SIZE = 4
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.repeat()

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

# with tf.Session() as sess:
# #     실행하면 무한 loop 진행
#     while True:
#         print(sess.run(el))

## Shuffle
 - shuffle을 통해 데이터를 섞는다. overfitting을 막기위해 중요한 기능이다.
 - buffer_size를 지정하여 uniform하게 선택되도록 한다. (seed와 유사)

In [31]:
# SHUFFLE
BATCH_SIZE = 4
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

[[1]
 [4]
 [3]
 [2]]


## MAP
 - map을 활용하여, 2개의 element를 2로 곱하는 예를 들어보자

In [32]:
# MAP
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.map(lambda x: x*2)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
#     this will run forever
        for _ in range(len(x)):
            print(sess.run(el))

[2]
[4]
[6]
[8]
